In [ ]:
#!/usr/bin/python3

import cv2
import numpy as np

from OpenGL.GLU import *
from OpenGL import *
from OpenGL.GLUT import *

from PIL import Image

from objloader import *
import matplotlib.pyplot as plt

import time




#import OpenGL.GLU as glu
#import OpenGL.GL as gl
#import OpenGL.GLUT as glut
#import cv2

## 1. Calibração da Camera

Feita via "Camera Calibration Toolbox for Matlab" de Jean-Yves Bouguet (http://www.vision.caltech.edu/bouguetj/calib_doc/)

Capturei frames 4 aleátorios do vídeo disponibilizado para fazer a calibração

A seguir, os parametros obtidos e suas respectivas incertezas:

Focal Length:          fc = [ 823.02785   857.55218 ] +/- [ 27.59011   25.29775 ]

Principal point:       cc = [ 566.92369   396.42114 ] +/- [ 12.48074   22.10106 ]

Skew:             alpha_c = [ 0.00000 ] +/- [ 0.00000  ]   => angle of pixel axes = 90.00000 +/- 0.00000 degrees

Distortion:            kc = [ 0.09076   -0.21946   -0.00693   -0.00024  0.00000 ] +/- [ 0.07284   0.50725   0.00776   0.00641  0.00000 ]

Pixel error:          err = [ 0.37915   0.42384 ]

Com essa saída definimos a matriz de parametros intrinsecos da câmera:

In [ ]:
intrinsicMatrix = np.array([
                            [823.02785, 0.0, 320], 
                            [0.0, 857.55218, 240], 
                            [0.0, 0.0, 1.0]
                            ])
cameraDistortion = [ 0.09076, -0.21946 ,-0.00693 ,-0.00024, 0.00000 ] 
#print(intrinsicMatrix)


In [ ]:
render_pikachu = True
render_cubes = True

# 2. Determinando a posição e orientação do alvo

## 2.1 O Video

##### 2.1.1 leitura do vídeo de input via OpenCV

Ref: https://theailearner.com/2018/10/15/extracting-and-saving-video-frames-using-opencv-python/

Ref2: https://stackoverflow.com/questions/33311153/python-extracting-and-saving-video-frames

In [ ]:
inputVideo = cv2.VideoCapture('entrada.mp4')

##### 2.1.2 Captura e decodificação de frames

In [ ]:
frames = []
#success, frame = inputVideo.read()
success,image = inputVideo.read()
while success:
    frames.append(image)
    success, image = inputVideo.read()

## 2.2 - O alvo

##### 2.2.1 Leitura e binarização do alvo

In [ ]:
alvo0rot = cv2.imread('alvo.jpg', 0)
_, alvo0rot = cv2.threshold(alvo0rot, 127, 255, cv2.THRESH_BINARY)

##### 2.2.2 Rotações do alvo

Ref: https://www.geeksforgeeks.org/python-opencv-cv2-rotate-method/

In [ ]:
alvo1rot = cv2.rotate(alvo0rot, cv2.ROTATE_90_CLOCKWISE) 
alvo2rot = cv2.rotate(alvo1rot, cv2.ROTATE_90_CLOCKWISE) 
alvo3rot = cv2.rotate(alvo2rot, cv2.ROTATE_90_CLOCKWISE) 
alvos = [alvo0rot, alvo1rot,alvo2rot,alvo3rot]

### 2.3 - Extração de bordas e contornos

##### 2.3.1 - Extração de bordas (B&W -> Binarização -> Bordas)
Ref: https://docs.opencv.org/master/da/d22/tutorial_py_canny.html

In [ ]:
def binarize(image):
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binarizado = cv2.threshold(grayscale, 127, 255, cv2.THRESH_BINARY)
    return binarizado
    

In [ ]:
def extract_edges_given_binarized(binarizado):
    imageEdges = cv2.Canny(binarizado, 100, 200)
    return imageEdges
    

##### 2.3.2 Extração de contornos

Ref: https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_contours/py_contours_begin/py_contours_begin.html#contours-getting-started

In [ ]:
def extract_contours_given_edges(imageEdges):
    contorno, _  = cv2.findContours(imageEdges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contorno

## 2.4 - Identificação de quadrilateros na imagem

Ref: https://stackoverflow.com/questions/55169645/square-detection-in-image/
Ref2: https://stackoverflow.com/questions/61166180/detect-rectangles-in-opencv-4-2-0-using-python-3-7

In [ ]:
def find_squares_given_contours(edgesContours):
    squares = list()
    for contour in edgesContours:
        epsilon = 0.05*cv2.arcLength(contour,True)
        polygon = cv2.approxPolyDP(contour, epsilon, True)
        if len(polygon) == 4 and cv2.isContourConvex(polygon):
            squares.append(polygon)
    return squares

In [ ]:
def template_corners_coords(template):
    s = template.shape
    return np.float32([[0,0], [0, s[0]], [s[1], s[0]], [s[1], 0]])

In [ ]:
def homography_and_matching(corner_coords,quadrados, image):
    quadrados_matched = []
    orientacao = []
    poses_na_cena = []
    for q in quadrados: 
        image_homography, _ = cv2.findHomography(np.float32(q), corner_coords, cv2.RANSAC)
        result = cv2.warpPerspective(image, image_homography, alvo0rot.shape)           
        
        diffs = [0] * len(alvos)

        for i,rot in enumerate(alvos):
            diffs[i] = np.sum(np.abs(result - rot)) / (result.size)
            
        #Valor 50 para a diferença absoluta foi escolhido de forma empirica
        if min(diffs) <20:
            poses_na_cena.append( (q, diffs.index(min(diffs))) )
            #quadrados_matched.append(q)
            #orientacao.append(diffs.index(min(diffs)))
            
    return poses_na_cena

In [ ]:
def homography(corner_coords,quadrados, image):
    quadrados_matched = []
    orientacao = []
    #poses_na_cena = []
    for q in quadrados: 
        image_homography, _ = cv2.findHomography(np.float32(q), corner_coords, cv2.RANSAC)
        result = cv2.warpPerspective(image, image_homography, alvo0rot.shape)           
        
        diffs = [0] * len(alvos)
        for i,rot in enumerate(alvos):
            diffs[i] = np.sum(np.abs(result - rot)) / (result.size)
            
        #Valor 50 para a diferença absoluta foi escolhido de forma empirica
        if min(diffs) <20 :
            #poses_na_cena.append( (q, diffs.index(min(diffs))) )
            quadrados_matched.append(q)
            orientacao.append(diffs.index(min(diffs)))
            
    #return (quadrados_matched, orientacao)
    return (quadrados_matched, orientacao)
    

In [ ]:
def processa_frames(frames, alvo, mapa_rotacao):
    frame_processado = [] # Cada elemento é uma tripla (frame, posicao, orientacao)
    for frame in frames:
        binarizado = binarize(frame)
        edges = extract_edges_given_binarized(binarizado)
        contornos = extract_contours_given_edges(edges)

        # Aproxima um poligono em cada contorno obtido. Caso tal polígono possua quatro lados e seja
        # convexo, então o mesmo é classificado como quadrilatero - que é a mesma classificação dos alvos
        # procurados.
        lista_quadrilateros = find_squares_given_contours(contornos)
    
        
        
        cornerCoords = template_corners_coords(alvo0rot)
        
    

        frame_processado.append(homography_and_matching(cornerCoords, lista_quadrilateros, binarizado))
        

    return frame_processado

In [ ]:
cornerCoords = template_corners_coords(alvo0rot)
#videoMatches = list()
poly = list()
#matchesOrientations = list()
direction = list()

cena = []

for frame in frames:
    binarizado = binarize(frame)
    edges = extract_edges_given_binarized(binarizado)
    contorno = extract_contours_given_edges(edges)
    squares = find_squares_given_contours(contorno)
    temp1,temp2 = homography(cornerCoords,squares, binarizado)
    #videoMatches.append(frameMatches)
    poly.append(temp1)
    direction.append(temp2)
    #cena.append(homo)

    

In [ ]:
# cornerCoords = template_corners_coords(alvo0rot)
# #videoMatches = list()
# #poly = list()
# #matchesOrientations = list()
# #direction = list()

# cena = []

# for frame in frames:
#     binarizado = binarize(frame)
#     edges = extract_edges_given_binarized(binarizado)
#     contorno = extract_contours_given_edges(edges)
#     squares = find_squares_given_contours(contorno)
#     homo = homography(cornerCoords,squares, binarizado)
#     #videoMatches.append(frameMatches)
#     cena.append(homo)

    

In [ ]:
#highlight = cv2.drawContours(frames[120], videoMatches[120], -1, (100,100,255), 3)
#plt.imshow(highlight)

# 3. Os parâmetros extrínsecos

In [ ]:
#poly = [c[0] for c in cena]

In [ ]:
#direction = [c[1] for c in cena]

In [ ]:
def obtem_pose(polygon, direction):    
    #coords = list()   
    #coords.append(np.float32([[-1, 1, 0], [-1, -1, 0], [1, -1, 0], [1, 1, 0]])) # 0°
    #coords.append(np.float32([[-1, -1, 0], [1, -1, 0], [1, 1, 0], [-1, 1, 0]])) # 90°
    #coords.append(np.float32([[1, -1, 0], [1, 1, 0], [-1, 1, 0], [-1, -1, 0]])) # 180°
    #coords.append(np.float32([[1, 1, 0], [-1, 1, 0], [-1, -1, 0], [1, -1, 0]]))
    
    if direction == 0:
        dst = np.float32([[-1, 1, 0], [-1, -1, 0], [1, -1, 0], [1, 1, 0]])
    if direction == 1:
        dst = np.float32([[-1, -1, 0], [1, -1, 0], [1, 1, 0], [-1, 1, 0]])
    if direction == 2:
        dst = np.float32([[1, -1, 0], [1, 1, 0], [-1, 1, 0], [-1, -1, 0]])     
    if direction == 3:
        dst = np.float32([[1, 1, 0], [-1, 1, 0], [-1, -1, 0], [1, -1, 0]])

    _, rot, trans = cv2.solvePnP(dst, np.float32(polygon), intrinsicMatrix, np.float32(cameraDistortion))
    rodRot, _ = cv2.Rodrigues(rot)
    
    
    matriz_pose = np.append(rodRot, trans, axis=1)
    #lastRow = [0,0,0,1]
    matriz_pose = np.append(matriz_pose, [[0,0,0,1]], axis = 0)
    
       
    matriz_pose[1, 0] =  matriz_pose[1, 0] * -1
    matriz_pose[2, 0] = matriz_pose[2, 0] * -1 
    matriz_pose[1, 1] = matriz_pose[1, 1] * -1
    matriz_pose[2, 1] = matriz_pose[2, 1] * -1
    matriz_pose[1, 2] = matriz_pose[1, 2] * -1
    matriz_pose[2, 2] = matriz_pose[2, 2] * -1
    matriz_pose[1, 3] = matriz_pose[1, 3] * -1
    matriz_pose[2, 3] = matriz_pose[2, 3] * -1
    
    return np.transpose(matriz_pose)

In [ ]:
# Função para definir ajustes intrínsecos da câmera
def ajuste_intrinsecos_camera(width, height, K):
    fx = K[0, 0]
    fy = K[1, 1]
    fovy = 2 * np.arctan(0.5 * height / fy) * 180 / np.pi
    aspect = width * fy / (height * fx)
    gluPerspective(fovy, aspect, 0.1, 100.0)

In [ ]:
# Função de renderização dos cubos
def render_cube():
    glPushAttrib(GL_CURRENT_BIT)
    glColor3f(100/255,100/255,1)
    glLineWidth(3.0)
    glTranslate(0, 0, 1)
    glutWireCube(2.0)

    glLineWidth(4.0)
    glColor3f(1,100/255,100/255)
    glBegin(GL_LINES)
    glVertex3f(-1, 1, -0.9)
    glVertex3f(1, 1, -0.9)
    glEnd()

    glPopAttrib()

In [ ]:
# Renderizando cubos e pikachu
def render():
    global render_pikachu
    global render_cubes
    for p, d in zip(poly[current_frame], direction[current_frame]):
        m_mat = obtem_pose(p, d)
        glLoadMatrixf(m_mat)
        
        if render_cubes:
            render_cube()    
        if render_pikachu:
            glCallList(pikachu.gl_list)

In [ ]:
# Iniciando o OpenGL
def init_open_gl():
    glClearColor(0, 0, 0, 0)
    glClearDepth(1.0)
    glEnable(GL_DEPTH_TEST)
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    ajuste_intrinsecos_camera(640, 480, intrinsicMatrix)

In [ ]:
# Gerando o background
def background(img):
    bitmap_tex = glGenTextures(1)
    glBindTexture(GL_TEXTURE_2D, bitmap_tex)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_NEAREST)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_NEAREST)
    
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, 640, 480, 0, GL_BGR, GL_UNSIGNED_BYTE, img)

    glDepthMask(GL_FALSE)
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    glOrtho(0, 640, 480, 0.0, 0.0, 1)
    
    glEnable( GL_TEXTURE_2D )
    glBegin(GL_QUADS)
    glTexCoord2f(0, 0);
    glVertex2f(0, 0);
    glTexCoord2f(1, 0);
    glVertex2f(640, 0);
    glTexCoord2f(1, 1);
    glVertex2f(640, 480);
    glTexCoord2f(0, 1);
    glVertex2f(0, 480);
    glEnd()

    glBindTexture(GL_TEXTURE_2D, 0)
    glDepthMask(GL_TRUE)

In [ ]:
current_frame = 0

def display_callback(pikachu):
    global current_frame

    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

    background(frames[current_frame])
    current_frame = (current_frame + 1) % len(poly)

    glMatrixMode(GL_PROJECTION)
    glLoadIdentity() 
    ajuste_intrinsecos_camera(640, 480, intrinsicMatrix)

    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    glEnable(GL_TEXTURE_2D)

    render()
    
    glutSwapBuffers()
    time.sleep(1 / 30)

In [ ]:
dimensoes = (640, 480)
glutInit()
glutInitDisplayMode(GLUT_RGBA | GLUT_DOUBLE)
glutSetOption(GLUT_ACTION_ON_WINDOW_CLOSE, GLUT_ACTION_CONTINUE_EXECUTION)
glutInitWindowSize(*dimensoes)
glutCreateWindow(b'TP 2')

init_open_gl()

In [ ]:
pikachu = OBJ('Pikachu.obj', swapyz=True)
display = lambda : display_callback(pikachu)
glutDisplayFunc(display)
glutIdleFunc(glutPostRedisplay)

glutMainLoop()

In [ ]:
# Função para definir ajustes intrínsecos da câmera
def ajuste_intrinsecos_camera(width, height, K):
    fx = K[0, 0]
    fy = K[1, 1]
    fovy = 2 * np.arctan(0.5 * height / fy) * 180 / np.pi
    aspect = width * fy / (height * fx)
    gluPerspective(fovy, aspect, 0.1, 100.0)

In [ ]:
# Função de renderização dos cubos
def render_cube():
    glPushAttrib(GL_CURRENT_BIT)
    glColor3f(100/255,100/255,1)
    glLineWidth(3.0)
    glTranslate(0, 0, 1)
    glutWireCube(2.0)

    glLineWidth(4.0)
    glColor3f(1,100/255,100/255)
    glBegin(GL_LINES)
    glVertex3f(-1, 1, -0.9)
    glVertex3f(1, 1, -0.9)
    glEnd()

    glPopAttrib()

In [ ]:
# Renderizando cubos e pikachu
def render():
    coords = list()   
    coords.append(np.float32([[-1, 1, 0], [-1, -1, 0], [1, -1, 0], [1, 1, 0]])) # 0°
    coords.append(np.float32([[-1, -1, 0], [1, -1, 0], [1, 1, 0], [-1, 1, 0]])) # 90°
    coords.append(np.float32([[1, -1, 0], [1, 1, 0], [-1, 1, 0], [-1, -1, 0]])) # 180°
    coords.append(np.float32([[1, 1, 0], [-1, 1, 0], [-1, -1, 0], [1, -1, 0]]))
    global render_pikachu
    global render_cubes
    for p, d in zip(videoMatches[current_frame], matchesOrientations[current_frame]):
        m_mat = obtem_pose(p, d)
        glLoadMatrixf(m_mat)
        
        if render_cubes:
            render_cube()    
        if render_pikachu:
            glCallList(pikachu.gl_list)

In [ ]:
# Iniciando o OpenGL
def init_open_gl():
    glClearColor(0, 0, 0, 0)
    glClearDepth(1.0)
    glEnable(GL_DEPTH_TEST)
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    ajuste_intrinsecos_camera(640, 480, intrinsicMatrix)

In [ ]:
# Gerando o background
def background(img):
    bitmap_tex = glGenTextures(1)
    glBindTexture(GL_TEXTURE_2D, bitmap_tex)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_NEAREST)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_NEAREST)
    
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, 640, 480, 0, GL_BGR, GL_UNSIGNED_BYTE, img)

    glDepthMask(GL_FALSE)
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    glOrtho(0, 640, 480, 0.0, 0.0, 1)
    
    glEnable( GL_TEXTURE_2D )
    glBegin(GL_QUADS)
    glTexCoord2f(0, 0);
    glVertex2f(0, 0);
    glTexCoord2f(1, 0);
    glVertex2f(640, 0);
    glTexCoord2f(1, 1);
    glVertex2f(640, 480);
    glTexCoord2f(0, 1);
    glVertex2f(0, 480);
    glEnd()

    glBindTexture(GL_TEXTURE_2D, 0)
    glDepthMask(GL_TRUE)

In [ ]:
current_frame = 0

def display_callback(pikachu):
    global current_frame

    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

    background(frames[current_frame])
    current_frame = (current_frame + 1) % len(videoMatches)

    glMatrixMode(GL_PROJECTION)
    glLoadIdentity() 
    ajuste_intrinsecos_camera(640, 480, intrinsicMatrix)

    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    glEnable(GL_TEXTURE_2D)

    #render()
    
    glutSwapBuffers()
    time.sleep(1 / 30)

In [ ]:
dimensoes = (640, 480)
glutInit()
glutInitDisplayMode(GLUT_RGBA | GLUT_DOUBLE)
glutSetOption(GLUT_ACTION_ON_WINDOW_CLOSE, GLUT_ACTION_CONTINUE_EXECUTION)
glutInitWindowSize(*dimensoes)
glutCreateWindow(b'TP 2')

init_open_gl()

In [ ]:
pikachu = OBJ('Pikachu.obj', swapyz=True)
display = lambda : display_callback(pikachu)
glutDisplayFunc(display)
glutIdleFunc(glutPostRedisplay)

glutMainLoop()

In [ ]:
#OOLLDDDDD functional

In [ ]:
def initOpenGL(dimensions):
    width, height = dimensions

    gl.glClearColor(0.0, 0.0, 0.0, 0.0)
    gl.glClearDepth(1.0)

    gl.glEnable(gl.GL_DEPTH_TEST)

    gl.glMatrixMode(gl.GL_PROJECTION)
    gl.glLoadIdentity()
    
    
    fx = intrinsicMatrix[0, 0]
    fy = intrinsicMatrix[1, 1]
    fovy = 2*np.arctan(0.5*height/fy) * 180/np.pi
    aspect = (width*fy)/(height*fx)

    near = 0.1
    far = 100.0

    glu.gluPerspective(fovy, aspect, near, far)

In [ ]:
def resize_view(width, height, K):
    gl.glViewport(0, 0, width, height)

    gl.glMatrixMode(gl.GL_PROJECTION)
    gl.glLoadIdentity()
    
    fx = K[0, 0]
    fy = K[1, 1]
    fovy = 2*np.arctan(0.5*height/fy) * 180/np.pi
    aspect = (width*fy)/(height*fx)
    near = 0.1
    far = 100.0
    glu.gluPerspective(fovy, aspect, near, far)
    

    gl.glMatrixMode(gl.GL_MODELVIEW)
    gl.glLoadIdentity()
    glu.gluLookAt(0, 0, 5, 0, 0, 0, 0, 1, 0)

In [ ]:
def background(img):
    id_textura = gl.glGenTextures(1)
    gl.glBindTexture(gl.GL_TEXTURE_2D, id_textura)

    fundo = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #fundo = cv2.flip(fundo, 0)
    
    gl.glTexParameteri(gl.GL_TEXTURE_2D, gl.GL_TEXTURE_MAG_FILTER, gl.GL_LINEAR)
    gl.glTexParameteri(gl.GL_TEXTURE_2D, gl.GL_TEXTURE_MIN_FILTER, gl.GL_LINEAR)
    gl.glTexImage2D(gl.GL_TEXTURE_2D, 0, gl.GL_RGB, 640, 480, 0, gl.GL_RGB, gl.GL_UNSIGNED_BYTE, fundo)

    gl.glDepthMask(gl.GL_FALSE)
    gl.glDisable( gl.GL_DEPTH_TEST )

    gl.glMatrixMode(gl.GL_PROJECTION)
    gl.glLoadIdentity()
    gl.glOrtho(0, 640, 480, 0.0, 0.0, 1)
    
    gl.glEnable( gl.GL_TEXTURE_2D )
    gl.glBegin(gl.GL_QUADS)
    gl.glTexCoord2f(0, 0); gl.glVertex2f(0, 0)
    gl.glTexCoord2f(1, 0); gl.glVertex2f(640, 0)
    gl.glTexCoord2f(1, 1); gl.glVertex2f(640, 480)
    gl.glTexCoord2f(0, 1); gl.glVertex2f(0, 480)
    gl.glEnd()

    gl.glBindTexture(gl.GL_TEXTURE_2D, 0)
    gl.glEnable( gl.GL_DEPTH_TEST )
    gl.glDepthMask(gl.GL_TRUE)

    gl.glFlush()

In [ ]:
def desenha_pikachus_nas_posicoes(obj, infos_cena_atual):
    for posicao, orientacao in infos_cena_atual:   
        m = obtem_pose(posicao, orientacao)
        gl.glLoadMatrixf(m)
        gl.glCallList(obj.gl_list)

In [ ]:
def cria_e_posiciona_cubo(m):

    vertices = (
        (1, -1, -1), (1, 1, -1), (-1, 1, -1),
        (-1, -1, -1),(1, -1, 1), (1, 1, 1),
        (-1, -1, 1), (-1, 1, 1)
    )

    arestas = (
        (2,1), (0,1), (0,3),
        (0,4), (2,3), (2,7),
        (6,3), (6,4), (6,7),
        (5,1), (5,4), (5,7)
    )
    
    gl.glMatrixMode(gl.GL_MODELVIEW)
    gl.glLoadIdentity()
    gl.glLoadMatrixf(m)

    gl.glPushAttrib(gl.GL_CURRENT_BIT)
    gl.glLineWidth(4)

    gl.glBegin(gl.GL_LINES)
    gl.glColor3f(124/255, 252/255, 0)
    first_seg = True
    for aresta in arestas:
        for vertice in aresta:
            gl.glVertex3fv(vertices[vertice])
            if first_seg:
                first_seg = False
                gl.glColor3f(124/255, 252/255, 0)
            else:
                gl.glColor3f(230/255, 0., 126/255)
    gl.glEnd()

    gl.glMatrixMode(gl.GL_PROJECTION)
    gl.glLoadIdentity()
    gl.glPopAttrib()
    gl.glFlush()

In [ ]:
def desenha_cubos_nas_posicoes(infos_cena_atual):
    for posicao, orientacao in infos_cena_atual:   
        m = obtem_pose(posicao, orientacao)
        cria_e_posiciona_cubo(m)

In [ ]:
def renderiza_pikachus(infos_cena_atual, modelo_objeto):    
    gl.glMatrixMode(gl.GL_MODELVIEW)
    gl.glLoadIdentity()
    gl.glEnable(gl.GL_TEXTURE_2D)

    desenha_pikachus_nas_posicoes(modelo_objeto, infos_cena_atual)

In [ ]:
def renderiza_cubos(infos_cena_atual):    
    gl.glMatrixMode(gl.GL_MODELVIEW)
    gl.glLoadIdentity()
    gl.glEnable(gl.GL_TEXTURE_2D)

    desenha_cubos_nas_posicoes(infos_cena_atual)

In [ ]:
def cria_cena(frames, frames_processados, modelo_pikachu):
    global frame_index
    global render_cubo
    global render_pikachu

    if frame_index >= len(frames_processados) and not render_pikachu:
        render_pikachu = True
        render_cubo = False
        frame_index = 0

    elif frame_index >= len(frames_processados) and render_pikachu:
        return

    gl.glMatrixMode(gl.GL_MODELVIEW)
    gl.glLoadIdentity()
    gl.glClear(gl.GL_COLOR_BUFFER_BIT | gl.GL_DEPTH_BUFFER_BIT)

    infos_cena_atual = frames_processados[frame_index]

    background(frames[frame_index])
    gl.glMatrixMode(gl.GL_PROJECTION)
    gl.glLoadIdentity() 
    K = intrinsicMatrix
    
    
    
    fx = K[0, 0]
    fy = K[1, 1]

    fovy = 2*np.arctan(0.5*480/fy) * 180/np.pi
    aspect = (640*fy)/(480*fx)

    near = 0.1
    far = 100.0

    glu.gluPerspective(fovy, aspect, near, far)
    
    
    

    if render_pikachu:
        renderiza_pikachus(infos_cena_atual, modelo_pikachu)
    else:
        renderiza_cubos(infos_cena_atual)

    glut.glutSwapBuffers()

    time.sleep(1/18) # Valor qualquer pro vídeo nao ficar muito rapido
    frame_index = frame_index + 1

In [ ]:
def idleCallback():
    glut.glutPostRedisplay()

In [ ]:
def realidade_aumentada_pikachu(frames, frames_processados):
    dimensoes_frame = (640, 480)

    K = intrinsicMatrix

    glut.glutInit()
    glut.glutInitDisplayMode(glut.GLUT_RGBA | glut.GLUT_DOUBLE)
    glut.glutSetOption(glut.GLUT_ACTION_ON_WINDOW_CLOSE, glut.GLUT_ACTION_CONTINUE_EXECUTION)
    glut.glutInitWindowSize(*dimensoes_frame)
    janela = glut.glutCreateWindow(b'Realidade Aumentada - Pikachu')
    
    initOpenGL(dimensoes_frame)

    global frame_index
    frame_index = 0

    global render_pikachu
    render_pikachu = False

    global render_cubo
    render_cubo = True

    obj = OBJ("Pikachu.obj", swapyz=True)
    
    display_callback = lambda: cria_cena(frames, frames_processados, obj)
    reshape_callback = lambda w, h: resize_view(w, h, K)

    glut.glutDisplayFunc(display_callback)
    glut.glutReshapeFunc(reshape_callback)
    glut.glutIdleFunc(idleCallback)
    glut.glutMainLoop()

In [ ]:
#testando =(videoMatches,matchesOrientations)

In [ ]:
#testando = [[a]  for a in zip(videoMatches,matchesOrientations)]

In [ ]:
#testando

In [ ]:
#zip(videoMatches,matchesOrientations)

In [ ]:
alvo, cantos_alvo = alvo0rot, template_corners_coords(alvo0rot)

mapa_rotacao = {}
mapa_rotacao[0] = alvo0rot
mapa_rotacao[90] = alvo1rot
mapa_rotacao[180] = alvo2rot
mapa_rotacao[270] = alvo3rot

mapa_coordenada = {}
mapa_coordenada[90] = np.float32([[-1, 1, 0], [-1, -1, 0], [1, -1, 0], [1, 1, 0]])
mapa_coordenada[0] = np.float32([[-1, -1, 0], [1, -1, 0], [1, 1, 0], [-1, 1, 0]])
mapa_coordenada[270] = np.float32([[1, -1, 0], [1, 1, 0], [-1, 1, 0], [-1, -1, 0]])
mapa_coordenada[180] = np.float32([[1, 1, 0], [-1, 1, 0], [-1, -1, 0], [1, -1, 0]])



frames_processados = processa_frames(frames, alvo, mapa_rotacao)

In [ ]:
frames_processados

In [ ]:
realidade_aumentada_pikachu(frames, frames_processados)

In [ ]:
alvo, cantos_alvo = obtem_alvo()
frames = captura_frames()
mapa_rotacao, mapa_coordenada = gera_mapa_rotacoes_alvo(alvo)
frames_processados = processa_frames(frames, alvo, mapa_rotacao)

In [ ]:
def initOpenGL(dimensions):

    (width, height) = dimensions
    
    glClearColor(0.0, 0.0, 0.0, 0.0)
    glClearDepth(1.0)

    glEnable(GL_DEPTH_TEST)

    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
 
    fovy = 45
    aspect = (width)/(height)
    gluPerspective(fovy, aspect, 0.1, 100.0)

In [ ]:
def object3D(obj):

    # translada o objeto para ficar 10 unidades distante da camera (para podermos ver o objeto)
    glTranslate(0,0,-10)

    # move o model em y para centralizar ele
    glTranslate(0,-2,0)
    # rotaciona o modelo para podermos ve-lo de frente
    glRotate(90,1,0,0)
    glRotate(180,0,1,0)
    # renderiza o modelo do Pikachu
    glCallList(obj.gl_list)

    # renderiza um cubo
    # glutWireCube(2.0)

In [ ]:
def displayCallback():

    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    # carregar o modelo 3D do Pikachu
    obj = OBJ("Pikachu.obj", swapyz=True)

    # habilita o uso de texturas (o Pikachu tem textura)
    glEnable(GL_TEXTURE_2D)

    object3D(obj) 
        
    glutSwapBuffers() 

In [ ]:
def idleCallback():

    glutPostRedisplay()

In [ ]:
if __name__ == '__main__':
    
    dimensions = (640, 480)
    glutInit()
    glutInitDisplayMode(GLUT_RGBA | GLUT_DOUBLE)
    glutSetOption(GLUT_ACTION_ON_WINDOW_CLOSE, GLUT_ACTION_CONTINUE_EXECUTION)
    glutInitWindowSize(*dimensions)
    window = glutCreateWindow(b'Realidade Aumentada [codigo esqueleto]')
    
    initOpenGL(dimensions)
    
    glutDisplayFunc(displayCallback)
    glutIdleFunc(idleCallback)
    
    glutMainLoop()